<a href="https://colab.research.google.com/github/ffviana/NUTRECON/blob/main/FractalPairing_Pilot/Data%20Acquisition/Colab%20Notebooks/SequenceGen_ExpRoom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from random import shuffle
from datetime import datetime
from itertools import chain,cycle
from IPython.display import display_html

base_folder = '/content/drive/MyDrive/Food Reward and Behaviour/Humans/Projects/2. NUTRECON/FlavorFractal optimization/1. DataAcquisition/ExperimentsRoom'

sequenceFolder = '/SitmulusPresentationSequence'

saveDataFolder = '{}/{}'.format(base_folder, sequenceFolder)

def int_to_Roman(num):
  lookup = [
      (1000, 'M'),
      (900, 'CM'),
      (500, 'D'),
      (400, 'CD'),
      (100, 'C'),
      (90, 'XC'),
      (50, 'L'),
      (40, 'XL'),
      (10, 'X'),
      (9, 'IX'),
      (5, 'V'),
      (4, 'IV'),
      (1, 'I'),
  ]
  res = ''
  for (n, roman) in lookup:
      (d, num) = divmod(num, n)
      res += roman * d
  return res

def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Generate Flavour Presentation order

In [ ]:
Subject_number = 22  # CHANGE HERE

#===============================================================================

Subject_code = 'ercffa_{:03d}'.format(Subject_number)

flavor_code = {
    'a': 'Apple Cider',
    'b': 'Black Currant',
    'c': 'Cashew',
    'd': 'Lychee',
    'e': 'Pomegranate',
    'f': 'Tamarind',
    'g': 'Plain'
}

FlavorPresentationOrder = ['a','b','c','d','e','f']
shuffle(FlavorPresentationOrder)
presentationOrder_df = pd.DataFrame()

for p in range(len(FlavorPresentationOrder)):
  tmp = pd.DataFrame([p+1, 
                      flavor_code[FlavorPresentationOrder[p]],
                      FlavorPresentationOrder[p]]).T
  presentationOrder_df = pd.concat([presentationOrder_df, tmp])

presentationOrder_df.columns = ['Order', 'Flavor', 'Flavor_code']

print('\033[1mPresentation order:\033[0m')
presentationOrder_df = presentationOrder_df.set_index('Order')
presentationOrder_df = presentationOrder_df[['Flavor_code', 'Flavor']]


Flavors = ['a','f','e','c','b','d']   # CHANGE HERE

#===============================================================================

now = datetime.now()

current_time = now.strftime("%d-%m-%Y_%H-%M")

allFlavors = Flavors.copy()

Fractals = list(range(6))

#while len(allFlavors)< len(Fractals):
#  allFlavors.append('g')

shuffle(allFlavors)
shuffle(Fractals)
FractalFlavor_df = pd.DataFrame()
FractalFlavor_code = {}
for i in range(len(Fractals)):
  tmp = pd.DataFrame([Fractals[i],flavor_code[allFlavors[i]],allFlavors[i]]).T
  FractalFlavor_df = pd.concat([FractalFlavor_df, tmp], axis = 0)
  FractalFlavor_code[Fractals[i]] = flavor_code[allFlavors[i]]

FractalFlavor_df.columns = ['fractal', 'Flavor', 'Flavor_code']
FractalFlavor_df = FractalFlavor_df.sort_values(by='fractal')

learningOrder_df = FractalFlavor_df.sample(frac=1).reset_index(drop=True)
learningOrder_df.index = learningOrder_df.index + 1
learningOrder_df.index = learningOrder_df.index.rename('Order')
learningOrder_df['Fractal'] = [int_to_Roman(p + 1) for p in learningOrder_df['fractal']]
learningOrder_df = learningOrder_df[['Fractal', 'Flavor_code', 'Flavor']]

AssosFracs = FractalFlavor_df[FractalFlavor_df['Flavor'] != 'Plain']

allAssosFracs = pd.concat([AssosFracs]* 6, axis = 0)

AssocTest1 = allAssosFracs.sample(frac=1)
AssocTest2 = allAssosFracs.sample(frac=1)
AssocTest3 = allAssosFracs.sample(frac=1)

AssocTest1 = AssocTest1.reset_index(drop=True)
AssocTest1.index = AssocTest1.index + 1
AssocTest1.index = AssocTest1.index.rename('Order')

AssocTest2 = AssocTest2.reset_index(drop=True)
AssocTest2.index = AssocTest2.index + 1
AssocTest2.index = AssocTest2.index.rename('Order')

AssocTest3 = AssocTest3.reset_index(drop=True)
AssocTest3.index = AssocTest3.index + 1
AssocTest3.index = AssocTest3.index.rename('Order')

AssocTest1 = AssocTest1[['Flavor_code', 'Flavor', 'fractal']]
AssocTest2 = AssocTest2[['Flavor_code', 'Flavor', 'fractal']]
AssocTest3 = AssocTest3[['Flavor_code', 'Flavor', 'fractal']]

FractalFlavor_df['Fractal'] = [int_to_Roman(p + 1) for p in FractalFlavor_df['fractal']]
FractalFlavor_df = FractalFlavor_df.drop(columns = 'fractal')
FractalFlavor_df = FractalFlavor_df.set_index('Fractal')
FractalFlavor_df = FractalFlavor_df[['Flavor_code', 'Flavor']]

Data_dict = {'Subject ID': Subject_code,
             'Timestamp': datetime.timestamp(now),
             'Presentation Order': presentationOrder_df,
             'Flavor-Fractal pairs': FractalFlavor_df,
             'Learning Order': learningOrder_df,
             'Association Test 1': AssocTest1,
             'Association Test 2': AssocTest2,
             'Association Test 3': AssocTest3
            }

data_dir = '{}/{}_{}.npy'.format(saveDataFolder, Subject_code, current_time)

np.save(data_dir, Data_dict)

display_side_by_side(FractalFlavor_df, 
                     learningOrder_df,
                     AssocTest1[['Flavor_code', 'Flavor']],
                     AssocTest2[['Flavor_code', 'Flavor']],
                     AssocTest3[['Flavor_code', 'Flavor']],
                     titles=['Flavor Fractal Pairs', 
                             'Learning Phase Order',
                             'Assciation Test 1',
                             'Assciation Test 2',
                             'Assciation Test 3']) 


Presentation order:


Flavor Fractal Pairs 
 
 
 
 Flavor_code 
 Flavor 
 
 
 Fractal 
 
 
 
 
 
 
 I 
 c 
 Cashew 
 
 
 II 
 a 
 Apple Cider 
 
 
 III 
 d 
 Lychee 
 
 
 IV 
 b 
 Black Currant 
 
 
 V 
 f 
 Tamarind 
 
 
 VI 
 e 
 Pomegranate 
 
 
 Learning Phase Order 
 
 
 
 Fractal 
 Flavor_code 
 Flavor 
 
 
 Order 
 
 
 
 
 
 
 
 1 
 VI 
 e 
 Pomegranate 
 
 
 2 
 III 
 d 
 Lychee 
 
 
 3 
 II 
 a 
 Apple Cider 
 
 
 4 
 V 
 f 
 Tamarind 
 
 
 5 
 IV 
 b 
 Black Currant 
 
 
 6 
 I 
 c 
 Cashew 
 
 
 Assciation Test 1 
 
 
 
 Flavor_code 
 Flavor 
 
 
 Order 
 
 
 
 
 
 
 1 
 b 
 Black Currant 
 
 
 2 
 f 
 Tamarind 
 
 
 3 
 f 
 Tamarind 
 
 
 4 
 c 
 Cashew 
 
 
 5 
 b 
 Black Currant 
 
 
 6 
 b 
 Black Currant 
 
 
 7 
 a 
 Apple Cider 
 
 
 8 
 c 
 Cashew 
 
 
 9 
 e 
 Pomegranate 
 
 
 10 
 b 
 Black Currant 
 
 
 11 
 d 
 Lychee 
 
 
 12 
 e 
 Pomegranate 
 
 
 13 
 e 
 Pomegranate 
 
 
 14 
 d 
 Lychee 
 
 
 15 
 f 
 Tamarind 
 
 
 16 
 e 
 Pomegranate 
 
 
 17 
 d 
 Lychee 
 
 
 18 
 d 
 Lychee 
 
 
 19 
 a 
 Apple Cider 
 
 
 20 
 c 
 Cashew 
 
 
 21 
 f 
 Tamarind 
 
 
 22 
 d 
 Lychee 
 
 
 23 
 a 
 Apple Cider 
 
 
 24 
 a 
 Apple Cider 
 
 
 25 
 f 
 Tamarind 
 
 
 26 
 c 
 Cashew 
 
 
 27 
 c 
 Cashew 
 
 
 28 
 f 
 Tamarind 
 
 
 29 
 b 
 Black Currant 
 
 
 30 
 b 
 Black Currant 
 
 
 31 
 e 
 Pomegranate 
 
 
 32 
 a 
 Apple Cider 
 
 
 33 
 c 
 Cashew 
 
 
 34 
 a 
 Apple Cider 
 
 
 35 
 e 
 Pomegranate 
 
 
 36 
 d 
 Lychee 
 
 
 Assciation Test 2 
 
 
 
 Flavor_code 
 Flavor 
 
 
 Order 
 
 
 
 
 
 
 1 
 b 
 Black Currant 
 
 
 2 
 d 
 Lychee 
 
 
 3 
 e 
 Pomegranate 
 
 
 4 
 a 
 Apple Cider 
 
 
 5 
 f 
 Tamarind 
 
 
 6 
 d 
 Lychee 
 
 
 7 
 a 
 Apple Cider 
 
 
 8 
 c 
 Cashew 
 
 
 9 
 f 
 Tamarind 
 
 
 10 
 e 
 Pomegranate 
 
 
 11 
 b 
 Black Currant 
 
 
 12 
 c 
 Cashew 
 
 
 13 
 c 
 Cashew 
 
 
 14 
 f 
 Tamarind 
 
 
 15 
 c 
 Cashew 
 
 
 16 
 b 
 Black Currant 
 
 
 17 
 d 
 Lychee 
 
 
 18 
 c 
 Cashew 
 
 
 19 
 a 
 Apple Cider 
 
 
 20 
 b 
 Black Currant 
 
 
 21 
 c 
 Cashew 
 
 
 22 
 e 
 Pomegranate 
 
 
 23 
 a 
 Apple Cider 
 
 
 24 
 d 
 Lychee 
 
 
 25 
 f 
 Tamarind 
 
 
 26 
 e 
 Pomegranate 
 
 
 27 
 b 
 Black Currant 
 
 
 28 
 e 
 Pomegranate 
 
 
 29 
 f 
 Tamarind 
 
 
 30 
 d 
 Lychee 
 
 
 31 
 a 
 Apple Cider 
 
 
 32 
 a 
 Apple Cider 
 
 
 33 
 e 
 Pomegranate 
 
 
 34 
 d 
 Lychee 
 
 
 35 
 f 
 Tamarind 
 
 
 36 
 b 
 Black Currant 
 
 
 Assciation Test 3 
 
 
 
 Flavor_code 
 Flavor 
 
 
 Order 
 
 
 
 
 
 
 1 
 b 
 Black Currant 
 
 
 2 
 d 
 Lychee 
 
 
 3 
 d 
 Lychee 
 
 
 4 
 b 
 Black Currant 
 
 
 5 
 c 
 Cashew 
 
 
 6 
 a 
 Apple Cider 
 
 
 7 
 b 
 Black Currant 
 
 
 8 
 f 
 Tamarind 
 
 
 9 
 a 
 Apple Cider 
 
 
 10 
 d 
 Lychee 
 
 
 11 
 c 
 Cashew 
 
 
 12 
 c 
 Cashew 
 
 
 13 
 f 
 Tamarind 
 
 
 14 
 b 
 Black Currant 
 
 
 15 
 c 
 Cashew 
 
 
 16 
 f 
 Tamarind 
 
 
 17 
 f 
 Tamarind 
 
 
 18 
 d 
 Lychee 
 
 
 19 
 e 
 Pomegranate 
 
 
 20 
 e 
 Pomegranate 
 
 
 21 
 a 
 Apple Cider 
 
 
 22 
 d 
 Lychee 
 
 
 23 
 f 
 Tamarind 
 
 
 24 
 b 
 Black Currant 
 
 
 25 
 e 
 Pomegranate 
 
 
 26 
 f 
 Tamarind 
 
 
 27 
 a 
 Apple Cider 
 
 
 28 
 b 
 Black Currant 
 
 
 29 
 e 
 Pomegranate 
 
 
 30 
 e 
 Pomegranate 
 
 
 31 
 d 
 Lychee 
 
 
 32 
 c 
 Cashew 
 
 
 33 
 a 
 Apple Cider 
 
 
 34 
 a 
 Apple Cider 
 
 
 35 
 c 
 Cashew 
 
 
 36 
 e 
 Pomegranate